## Debugging \Tau  -Grid for KP

In [1]:
using Distributions, Gadfly, Gurobi, JuMP

In [2]:
include("KPNormal.jl")
using KP

In [3]:
## Establish a clean setup
#srand(8675309)
n = 100
cs = rand(n) * 20   #about 10% of items fit
taus = 4. * rand(n)
sigs = 1./sqrt(taus)
tau0 = 3

thetas = randn(n) / sqrt(tau0)
zs = randn(n) .* sigs + thetas
noise = randn(n) .* sigs
xs = zs + noise
ys = zs - noise

zstar = KP.ideal_val(thetas, cs)

#naive x values
qs_naive = KP.q(cs, xs)
dot(qs_naive, ys)/n, dot(qs_naive, thetas)/n

(0.0695353719847883,0.045344855226254255)

In [ ]:
## try a non-standard set-up
n = 100
cs = rand(n) * 20
taus = 8. * rand(n)
sigs = 1./sqrt(taus)

thetas = ones(n)
thetas[1:2:n] = 2
zs = randn(n) .* sigs + thetas
noise = randn(n) .* sigs
xs = zs + noise
ys = zs - noise

zstar = KP.ideal_val(thetas, cs)

#naive x values
qs_naive = KP.q(cs, xs)
dot(qs_naive, ys)/n, dot(qs_naive, thetas)/n

In [ ]:
# ##Debugging.  Try to get the randon numbers corresponding to third iteration.
# srand(8675309)
# n_max = 100000
# n = 100
# tau0 = 1

# #costs and taus
# cs = rand(n_max) * 20   #about 10% of items fit
# taus = 8. * rand(n_max)
# sigs = 1./sqrt(taus)

# #burn two and gen one.
# thetas = randn(n_max) / sqrt(tau0)
# xs = randn(n_max) .* sigs + thetas
# ys = randn(n_max) .* sigs + thetas

# thetas = randn(n_max) / sqrt(tau0)
# xs = randn(n_max) .* sigs + thetas
# ys = randn(n_max) .* sigs + thetas

# thetas = randn(n_max) / sqrt(tau0)
# xs = randn(n_max) .* sigs + thetas
# ys = randn(n_max) .* sigs + thetas
# zs = .5 * (xs + ys)

# #reclip everyone for sanity
# thetas = thetas[1:n]
# cs = cs[1:n]
# taus = taus[1:n]
# xs = xs[1:n]
# ys = ys[1:n]
# zs = zs[1:n]

# zstar = KP.ideal_val(thetas, cs)

# #naive x values
# qs_naive = KP.q(cs, xs)
# dot(qs_naive, ys)/n, dot(qs_naive, thetas)/n

In [ ]:
# #Resort everyone to make life easier. 
# indx = sortperm(xs ./ cs, rev=true)
# taus = taus[indx]
# xs = xs[indx]
# thetas = thetas[indx]
# cs = cs[indx]
# [cs xs KP.q(cs, xs)]

In [ ]:
##compute the same along a linear grid
max_qs, vals, objs = KP.best_q_tau(cs, xs, taus, ys);
objs/= zstar

tau_grid = collect(linspace(0, maximum(vals)+1, 100))
tau_grid = append!(tau_grid, vals)
tau_grid = sort!(tau_grid)

outs = zeros(length(tau_grid))
for ix = 1:length(tau_grid)
    rs = KP.shrink(xs, taus, tau_grid[ix])
    q = KP.q(cs, rs)
    outs[ix] = dot(q, ys)/n
end

outs /= zstar

plot(layer(x=tau_grid, y=outs, Geom.line, Theme(default_color=color("black"))), 
layer(x=vals, y=objs, Geom.point)) 

In [ ]:
##compute the q_mle and qmm values to check they make sense
tau_mle, qs_mle = KP.q_MLE(cs, zs, taus)
dot(ys, qs_mle)/n, dot(thetas, qs_mle)/n


In [ ]:
tau_mm, qs_mm = KP.q_MM(cs, zs, taus)
dot(ys, qs_mm)/n, dot(thetas, qs_mm)/n

In [ ]:
# #perturb a little and compare
# xs2 = KP.shrink(xs, taus, 0)
# m2 = Model(solver=GurobiSolver(OutputFlag=false))
# @defVar(m2, 0 <= q2[1:n] <= 1)
# @addConstraint(m2, budget_constr, sum{cs[i]*q2[i], i=1:n}/ n <= 1 )
# @setObjective(m2, Max, sum{xs2[i] * q2[i], i=1:n}/ n )
# solve(m2)
# lambda2 = getDual(budget_constr)
# red_costs2 = [getDual(q2[ix]) for ix = 1:n]
# qs2 = getValue(q2)

# #println("Svals:\n", KP.sbars(xs, cs, taus, 6))


# [max_qs qs2 red_costs2]

# dot(thetas, qs2)/n, dot(cs, qs2)/n, lambda2

In [ ]:
# ##  Solve a knapsack the stupid way to figure out what's going on.
# m = Model(solver=GurobiSolver(OutputFlag=false))
# @defVar(m, 0 <= q[1:n] <= 1)
# @addConstraint(m, budget_constr, sum{cs[i]*q[i], i=1:n}/ n <=1  )
# @setObjective(m, Max, sum{xs[i] * q[i], i=1:n}/ n )
# solve(m)
# qs = getValue(q)
# red_costs = [getDual(q[ix]) for ix=1:n]

# lambda = getDual(budget_constr)

# ##use gurobi's feature to try and pick out the inner thing
# mpb = getInternalModel(m) # MathProgBase level model object
# grb = MathProgBase.getrawsolver(mpb) # Gurobi level model object
# min_obj_coeffs = Gurobi.get_dblattrarray(grb, "SAObjLow", 1, MathProgBase.numvar(m))

# [cs xs KP.q(cs, xs) qs]

In [4]:
KP.test1("temp", 2, [100, 1000, 10000, 100000], 8675309, 1)

elapsed time: 0.00011863 seconds
elapsed time: 6.9242e-5 seconds
elapsed time: 8.6982e-5 seconds
elapsed time: 7.3697e-5 seconds
elapsed time: 0.100122899 seconds
elapsed time: 7.4213e-5 seconds
elapsed time: 0.012438636 seconds
elapsed time: 0.000150658 seconds
elapsed time: 0.000120948 seconds
elapsed time: 0.000169752 seconds
elapsed time: 0.000381363 seconds
elapsed time: 0.000219228 seconds
elapsed time: 0.000259041 seconds
elapsed time: 0.008081531 seconds
elapsed time: 0.000320136 seconds
elapsed time: 0.002381345 seconds
elapsed time: 0.002499744 seconds
elapsed time: 0.003830694 seconds
elapsed time: 0.002607408 seconds
elapsed time: 0.002807523 seconds
elapsed time: 0.0023699 seconds
elapsed time: 0.002704565 seconds
elapsed time: 0.122078206 seconds
elapsed time: 0.003500516 seconds
elapsed time: 0.139298024 seconds
elapsed time: 0.157789408 seconds
elapsed time: 0.137241491 seconds
elapsed time: 0.033520877 seconds
elapsed time: 0.028638925 seconds
elapsed time: 0.023576866

LoadError: LoadError: InterruptException:
while loading In[4], in expression starting on line 1